In [ ]:
import pandas as pd
import numpy as np

def calculate_optimal_pnl(df, heat_rate=10, mw=250, min_run_hours=3, min_down_hours=6):
    # Calculate potential PnL for each hour
    df['potential_pnl'] = (df['power_price'] - (df['gas_price'] * heat_rate)) * mw
    df['cumulative_pnl'] = df['potential_pnl'].cumsum()
    df['run_decision'] = False
    df['pnl'] = 0.0

    n = len(df)

    def find_profitable_operation(start):
        max_profit = 0
        optimal_run_length = 0
        for run_length in range(min_run_hours, n - start + 1):
            end = start + run_length
            if end + min_down_hours > n:  # Ensure we respect the end of the timeframe
                break
            period_profit = df['cumulative_pnl'].iloc[end - 1] - (df['cumulative_pnl'].iloc[start - 1] if start > 0 else 0)
            if period_profit > max_profit:
                max_profit = period_profit
                optimal_run_length = run_length
            # Check future profitability considering min_down_hours
            if end + min_down_hours < n:
                future_profit = df['cumulative_pnl'].iloc[end + min_down_hours - 1] - df['cumulative_pnl'].iloc[end - 1]
                if period_profit + future_profit > max_profit:
                    break  # Future profitability justifies running through unprofitable hours
        return optimal_run_length, max_profit

    current_time = 0
    while current_time < n:
        run_length, profit = find_profitable_operation(current_time)
        if run_length > 0:
            # Mark the decision to run and calculate PnL
            df.loc[df.index[current_time]:df.index[current_time + run_length - 1], 'run_decision'] = True
            df.loc[df.index[current_time]:df.index[current_time + run_length - 1], 'pnl'] = \
                df.loc[df.index[current_time]:df.index[current_time + run_length - 1], 'potential_pnl']
            current_time += run_length + min_down_hours  # Account for min_down_hours after running
        else:
            current_time += 1  # Move to the next hour if not running

    # Cleanup: Remove the cumulative PnL column
    df.drop(columns=['cumulative_pnl'], inplace=True)
    return df

# Example DataFrame setup
data = {
    'time': pd.date_range(start='2022-01-01', periods=48, freq='H'),
    'power_price': np.random.uniform(20, 100, 48),  # Example power prices
    'gas_price': np.random.uniform(2, 5, 48),      # Example gas prices
}
df = pd.DataFrame(data)
df.set_index('time', inplace=True)

# Run the optimized function
optimized_df = calculate_optimal_pnl(df, heat_rate=10, mw=250, min_run_hours=3, min_down_hours=6)

# Display the optimized DataFrame
print(optimized_df[['power_price', 'gas_price', 'potential_pnl', 'run_decision', 'pnl']])

